In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import json
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

PATH

In [ ]:
MODEL_PATH = "best_model.pth" # ubah sesuai nama file modelmu
LABEL_MAP_PATH = "label_map.json"
TEST_DIR = "./Test"
OUTPUT_CSV = "jawaban.csv"

Load Label Map

In [ ]:
with open(LABEL_MAP_PATH, "r") as f:
label_map = json.load(f)


inv_label_map = {v: k for k, v in label_map.items()}


# ---- Transform ----
test_transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
])


# ---- Dataset & DataLoader ----
test_dataset = datasets.ImageFolder(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Load Model

In [ ]:
model = torch.load(MODEL_PATH, map_location=torch.device("cpu"))
model.eval()


all_preds = []
all_labels = []
file_names = []


# ---- Inference ----
with torch.no_grad():
for imgs, labels in test_loader:
outputs = model(imgs)
preds = torch.argmax(outputs, dim=1)


all_preds.extend(preds.tolist())
all_labels.extend(labels.tolist())

Confusion Matrix & Classification Report

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=list(label_map.keys()))


print("CONFUSION MATRIX:\n", cm)
print("\nCLASSIFICATION REPORT:\n", report)

Simpan CSV Jawaban

In [ ]:
predicted_labels = [inv_label_map[p] for p in all_preds]


# Ambil list file dari dataset
for path, _ in test_dataset.samples:
file_names.append(path.split('/')[-1])


csv_data = pd.DataFrame({
"filename": file_names,
"label": predicted_labels
})


csv_data.to_csv(OUTPUT_CSV, index=False)
print(f"CSV jawaban disimpan ke {OUTPUT_CSV}")